# Implementazione del task 5 -  Make Transactions

In questo task raggruppiamo gli eventi appartenenti allo stesso partecipante secondo una determinata condizione (ad esempio, lo stesso giorno). Infatti, a questo punto, gli eventi sono tutti nella forma $(pid, ts, label)$ e quindi possiamo creare un dataset transazionale raggruppando per giorno e selezionando $label$ come valore per ogni elemento.

Inoltre, se la $label$ presenta molti attributi è possibile ridurne il numero specificando quali tenere attraverso una funzione apposita.

In [1]:
# IMPORT
import nbimporter
import ETLBasics_t1 as task1
import ProfilingBasics_t2 as task2
import TimeSeriesToEvents_t3 as task3
import FeatureReduction_t4 as task4
import dateutil as du
from datetime import datetime
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance

import warnings
warnings.simplefilter("ignore", UserWarning)

In [2]:
# VARIABILI
PATH = './pmdata/'
people = [1,2,3]

In [3]:
# IPER-PARAMETRI

# raggruppamento task 3
K_GROUP_DAY = 1
K_GROUP_HOUR = 1

# attributi temporali per le label
times_attributes = ['weekend']

### Implementazione del task 1

In [4]:
sedentary_minutes = task1.sedentary_minutes_to_df(PATH, people)
sleep_0, sleep_1 = task1.sleep_to_df(PATH, people)
exercise_0, exercise_1 = task1.exercise_to_df(PATH, people)
lightly_active_minutes = task1.lightly_active_minutes_to_df(PATH, people)
time_in_heart_rate_zones = task1.time_in_heart_rate_zones_to_df(PATH, people)
moderately_active_minutes = task1.moderately_active_minutes_to_df(PATH, people)
very_active_minutes = task1.very_active_minutes_to_df(PATH, people)
resting_heart_rate = task1.resting_heart_rate_to_df(PATH, people)
srpe = task1.srpe_to_df(PATH, people)
wellness = task1.wellness_to_df(PATH, people)
injury = task1.injury_to_df(PATH, people)

try:
    steps = pd.read_pickle("dataframes/steps.pkl").loc[people]
    distance = pd.read_pickle("dataframes/distance.pkl").loc[people]
    calories = pd.read_pickle("dataframes/calories.pkl").loc[people]
    heart_rate = pd.read_pickle("dataframes/heart_rate.pkl").loc[people]
except:
    steps = task1.steps_to_df(PATH, people)
    distance = task1.distance_to_df(PATH, people)
    calories = task1.calories_to_df(PATH, people)
    heart_rate = task1.heart_rate_to_df(PATH, people)

### Implementazione dei task 3 e 4

In [5]:
group_time_series_map = {
    'DAY':  [sedentary_minutes,lightly_active_minutes,moderately_active_minutes,very_active_minutes,resting_heart_rate,time_in_heart_rate_zones],
    'HOUR': [calories]  #,distance,heart_rate,steps
}
event_name = [exercise_0,exercise_1]  #,sleep_0,sleep_1,srpe,wellness,injury

day_kmeans_data = []
hour_kmeans_data = []
event_kmeans_data = []

for df in group_time_series_map['DAY']:
    df = task3.from_ts_to_event_based_data(df,'DAY', K_GROUP_DAY) # questo andrà ad aggiungere per ogni colonna numerica _mean e _std
    df = task4.kmeans_discretization(df)
    day_kmeans_data.append(df)

for df in group_time_series_map['HOUR']:
    df = task3.from_ts_to_event_based_data(df,'HOUR', K_GROUP_HOUR)
    df = task4.kmeans_discretization(df)
    hour_kmeans_data.append(df)


for event in event_name:
    event = task4.kmeans_discretization(event, is_event=True)
    # estraggo le colonna TS per utilizzare la procedura che crea le transazioni
    # Viene estratto dall'indice TS e viene creata la corrispondente colonna TS
    event = event.reset_index(level='TS')
    event['obj_TS'] = [x for x in event['TS']]  # per evitare errori in caso di ricalcolo del task 5
    event_kmeans_data.append(event)

In [6]:
def extract_cluster_index_columns_from_df(df):
    '''
    Estraggo i nomi delle colonne che hanno come sottostringa _cluster_index.
    Poi si ritorna i nomi di quelle colonne
    '''
    col_names = list(df.columns)
    res = []
    for name in col_names:
        if '_cluster_center' in name:
            res.append(name)

    return res

def create_label(df, categorical_attributes, times_attributes):
    '''
    Crea le label per ogni riga del dataframe usando le colonne create dal kmeans, le colonne categoriche (se presenti)
    e gli attributi temporali (half_day, weekday ecc.)
    '''
    cluster_index_col_names = extract_cluster_index_columns_from_df(df)
    
    res = []
    for i in range(len(df)):
        tmp = ""
        for col_name in cluster_index_col_names + categorical_attributes + times_attributes:
            tmp += col_name + ": " + str(df.iloc[i][col_name]) + ", "
        
        res.append(tmp[:-2]) # -2 cosi' rimuoviamo l'ultimo ', '
    return res

def create_label_events(df, categorical_attributes, times_attributes):
    '''
    Crea le label per ogni riga di un dataframe evento usando le colonne create dal kmeans, le colonne categoriche (se presenti)
    e gli attributi temporali (week_day, weekend) calcolati tramite il timestamp
    '''
    
    df['DAY'] = df['obj_TS'].apply(lambda x: x.strftime("%Y-%m-%d"))
    df = df.copy()
    
    df['week_day'] = df['DAY'].apply(lambda x: du.parser.parse(x).strftime('%A'))
    df['weekend']= df['week_day'].apply(lambda x: True if x in ['Saturday', 'Sunday'] else False)
    
    cluster_index_col_names = extract_cluster_index_columns_from_df(df)
    
    res = []
    for i in range(len(df)):
        tmp = ""
        for col_name in cluster_index_col_names + categorical_attributes + times_attributes:
            tmp += col_name + ": " + str(df.iloc[i][col_name]) + ", "
        
        res.append(tmp[:-2]) # -2 cosi' rimuoviamo l'ultimo ', '
    return res

def make_transactions(day_kmeans_data, hour_kmeans_data, event_kmeans_data, times_attributes=[]):
    '''
    Questa funzione deve creare le transazioni mettendo insieme le label create da ogni tipo di dataframe. Le transazioni
    si ottengono raggrupando le singole righe in base al p_id (presente nell'indice dei df) e in base all'informazione DAY
    che è l'ultimo livello di raggruppamento. La colonna TS servirà nel task 7

    '''
    for df in day_kmeans_data:
        labels = create_label(df, [], times_attributes)
        df['label'] = labels
        tmp = [x for x in df['DAY']]
        df['TS'] = [datetime(int(x[0:4]), int(x[5:7]), int(x[8:10]), 0, 0, 0, 0).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3] for x in tmp]

    for df in hour_kmeans_data:
        labels = create_label(df, [], times_attributes)
        df['label'] = labels
        tmp = [x for x in df['HOUR']]
        df['TS'] = [datetime(int(x[0:4]), int(x[5:7]), int(x[8:10]), int(x[11:13]), 0, 0, 0).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3] for x in tmp]
        df['DAY'] = [x.split(" ")[0] for x in df['TS']]
    
    
    for df in event_kmeans_data:
        categorical_attributes = list(set(df.columns.values) - set(task2.get_continuous_attributes(df)) - set(['label','TS','DAY']))
        labels = create_label_events(df, categorical_attributes, times_attributes)
        df['label'] = labels
        # negli eventi è già presente TS perché la colonna è stata estratta dall'indice prima che la funzione 
        # venisse invocata
        df['TS'] = df['obj_TS'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S.000"))

    group_df = pd.concat(day_kmeans_data + hour_kmeans_data + event_kmeans_data)
    group_df = group_df.loc[:, ['TS', 'DAY', 'label']]
    group_df = group_df.sort_values(by=['TS'])

    group_df['t_item'] = group_df.apply(lambda x:   [x['TS'], x.label], axis = 1)
    group_df['p_id'] = [x if type(x)==int else x[0] for x in list(group_df.index)]
    transactions = list(group_df[['p_id', 'DAY', 't_item']].groupby(['p_id','DAY' ])['t_item'].apply(list))
    return transactions

## Esempio

L'insieme di transazioni che andremo a creare sarà una lista di liste (nel nostro caso, una per ogni giorno) possibilmente con lunghezze variabili, in quanto ottenute anche da dataframe che erano già in forma di evento.

In [8]:
transactions = make_transactions(day_kmeans_data, hour_kmeans_data, event_kmeans_data, times_attributes=times_attributes)
transactions[0]

[['2019-11-01 00:00:00.000',
  'sedentary_minutes_cluster_center: [626.67, 0.0], weekend: False'],
 ['2019-11-01 00:00:00.000',
  'calories_cluster_center: [1.32, 0.03], weekend: False'],
 ['2019-11-01 00:00:00.000',
  'resting_heart_rate_cluster_center: [53.7, 0.0], resting_heart_rate_error_cluster_center: [6.8, 0.0], weekend: False'],
 ['2019-11-01 00:00:00.000',
  'very_active_minutes_cluster_center: [72.5, 0.0], weekend: False'],
 ['2019-11-01 00:00:00.000',
  'lightly_active_minutes_cluster_center: [249.89, 0.0], weekend: False'],
 ['2019-11-01 00:00:00.000',
  'BELOW_DEFAULT_ZONE_1_cluster_center: [1241.44, 0.0], IN_DEFAULT_ZONE_1_cluster_center: [164.58, 0.0], IN_DEFAULT_ZONE_2_cluster_center: [2.3, 0.0], IN_DEFAULT_ZONE_3_cluster_center: [-0.0, 0.0], weekend: False'],
 ['2019-11-01 00:00:00.000',
  'moderately_active_minutes_cluster_center: [60.83, 0.0], weekend: False'],
 ['2019-11-01 01:00:00.000',
  'calories_cluster_center: [1.32, 0.03], weekend: False'],
 ['2019-11-01 02:0